# Cyclistic Bikeshare Case Study

## #Import the necessary libraries

In [1]:
import pandas as pd               # data manipulation and analysis
import numpy as np                # efficient data types
import matplotlib.pyplot as plt   # plotting visuals
import seaborn as sns             # visualization module
import glob                       # pathnames matching
import os                         # data file path handling
import datetime                   # For date manipulation
import math                       # For math functions
import h3                         # To calculate distance

In [2]:
# Import all csv files and concatenate them into one dataframe.

path = r'C:\Users\Victor Muange\Desktop\Cyclistic Case Study\Bikeshare dataset Apr-2020 to Mar-2021'
all_files = glob.glob(os.path.join(path, "*.csv"))
df_from_each_file = (pd.read_csv(f) for f in all_files)
Main_df = pd.concat(df_from_each_file, ignore_index = True)

## Exploring the data for cleaning.

In [3]:
# Prints the top five rows of the data
Main_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A847FADBBC638E45,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,86,Lincoln Ave & Diversey Pkwy,152.0,41.8964,-87.6610,41.9322,-87.6586,member
1,5405B80E996FF60D,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,503,Kosciuszko Park,499.0,41.9244,-87.7154,41.9306,-87.7238,member
2,5DD24A79A4E006F4,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,142,Indiana Ave & Roosevelt Rd,255.0,41.8945,-87.6179,41.8679,-87.6230,member
3,2A59BBDF5CDBA725,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,216,Wood St & Augusta Blvd,657.0,41.9030,-87.6975,41.8992,-87.6722,member
4,27AD306C119C6158,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,125,Sheridan Rd & Lawrence Ave,323.0,41.8902,-87.6262,41.9695,-87.6547,casual


In [4]:
# Check the number of raws in the dataframe.
len(Main_df.index)

3489384

In [5]:
# Get information from our dataframe (number of records, memory use and data types)
Main_df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489384 entries, 0 to 3489383
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 2.0 GB


In [6]:
# Get descriptive statistics under each numeric column
Main_df.describe().apply(lambda s: s.apply('{0:.3f}'.format))

,start_lat,start_lng,end_lat,end_lng
count,3489384.000,3489384.000,3484646.000,3484646.000
mean,41.904,-87.645,41.904,-87.645
std,0.044,0.026,0.044,0.026
min,41.640,-87.870,41.540,-88.070
25%,41.882,-87.659,41.883,-87.659
50%,41.900,-87.642,41.901,-87.643
75%,41.930,-87.628,41.931,-87.628
max,42.080,-87.520,42.160,-87.440


# A. Data cleaning

## Checking for duplicate values

In [7]:
# checking the number of unique values per column
print("Unique values per column")
for col in Main_df.columns:
    print(f"{col}: {Main_df[col].nunique()}")

Unique values per column
ride_id: 3489108
rideable_type: 3
started_at: 2205321
ended_at: 2196019
start_station_name: 708
start_station_id: 1344
end_station_name: 706
end_station_id: 1345
start_lat: 296482
start_lng: 291862
end_lat: 319770
end_lng: 307617
member_casual: 2


### Findig the duplicates for ride_id

In [8]:
def get_duplicates(field: str) -> pd.DataFrame:
    """Display rows with duplicate ride_ids"""
    vc = Main_df[field].value_counts()
    duplicate_index = vc[vc > 1].index.to_list()
    n_duplicates = len(duplicate_index)
    print(f"Number of duplicates for {field}: {n_duplicates}")
    if n_duplicates > 0:
        print("Sample rows:")
        mask = Main_df[field].isin(duplicate_index)
        return Main_df[mask].sort_values(by = field).head()
    return

get_duplicates('ride_id')

Number of duplicates for ride_id: 264
Sample rows:


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
2829219,021A73F8C18B932D,docked_bike,2020-11-25 16:35:39,2020-11-25 16:48:02,Clark St & Winnemac Ave,325.0,Ravenswood Ave & Berteau Ave,314.0,41.973347,-87.667855,41.957921,-87.673567,member
3006144,021A73F8C18B932D,docked_bike,12/15/2020 12:15,11/25/2020 16:48,Clark St & Winnemac Ave,TA1309000035,Ravenswood Ave & Berteau Ave,TA1309000018,41.973347,-87.667855,41.957921,-87.673567,member
3020020,0334987B57662109,docked_bike,12/15/2020 11:56,11/25/2020 16:22,Broadway & Berwyn Ave,13109,Lakefront Trail & Bryn Mawr Ave,KA1504000152,41.978353,-87.659753,41.984037,-87.652310,member
2797158,0334987B57662109,docked_bike,2020-11-25 16:15:04,2020-11-25 16:22:04,Broadway & Berwyn Ave,294.0,Lakefront Trail & Bryn Mawr Ave,459.0,41.978353,-87.659753,41.984037,-87.652310,member
3029795,038CAB4A84D9A56B,docked_bike,12/15/2020 11:57,11/25/2020 16:24,Wabash Ave & 9th St,TA1309000010,Wabash Ave & 9th St,TA1309000010,41.870769,-87.625734,41.870769,-87.625734,member


## Drop duplicate ride_id

In [9]:
Main_df = Main_df.drop_duplicates(subset=['ride_id'])

In [10]:
# Confirm that duplicate ride_ids were dropped
get_duplicates('ride_id')

Number of duplicates for ride_id: 0


# Check for NaN values

In [11]:
# Check for NaN values in start_station_id
Main_df['start_station_id'].isna().sum()

122723

## Drop NaN values in start_station_id

In [12]:
Main_df = Main_df.dropna(subset=['start_station_id']) 

In [13]:
#Confirm that NaN values were dropped.
Main_df['start_station_id'].isna().sum()

0

In [14]:
# Check for NaN values in end_station_id
Main_df['end_station_id'].isna().sum()

72208

## Drop NaN values in end_station_id

In [15]:
Main_df = Main_df.dropna(subset=['end_station_id'])

In [16]:
#Confirm that NaN values were dropped.
Main_df['end_station_id'].isna().sum()

0

# B. Data Manipulation.
### Adding the necessary columns

## 1. Add a ride_duration column 

In [ ]:
# convert time object to datetime

Main_df['started_at'] = Main_df['started_at'].astype('datetime64[ns]')
Main_df['ended_at'] = Main_df['ended_at'].astype('datetime64[ns]')

In [ ]:
# Adds ride_duration column
Main_df['ride_duration_mins'] = (((Main_df['ended_at'] - Main_df['started_at']))/pd.Timedelta(minutes=1))

### 2. Add a day column to show which day the ride starts

In [ ]:
Main_df['day'] = pd.to_datetime(Main_df['started_at']).dt.day_name()

### 3. Add a month_year column

In [ ]:
Main_df['month'] = pd.to_datetime(Main_df['started_at']).dt.strftime('%B-%Y')

### 4. Add a ride_distance column
#### Subtract start and end coordinates

In [ ]:
Main_df['ride_distance'] = Main_df.apply(lambda row: h3.point_dist((row['start_lat'], row['start_lng']), (row['end_lat'], row['end_lng']), unit='km'), axis=1)

In [ ]:
# Count number of rows where ride_distance is equal to or less than zero

(Main_df['ride_distance'] <=0).sum()  .sum()

In [ ]:
# Count total rows in Main_df
len(Main_df.index)

In [ ]:
# Drop the zero and negative vales in ride_distance
Main_df = Main_df.drop(Main_df[(Main_df['ride_distance'] <=0)].index)

In [ ]:
#Check that there are no more zero and negative values in ride_distance
(Main_df['ride_distance'] <=0).sum()  .sum()

In [ ]:
# Count total rows in Main_df
len(Main_df.index)

### 5. Add a start_hour column
It helps determine what time of day most riders get the bikes

In [ ]:
Main_df['start_hour'] = Main_df['started_at'].dt.hour

In [ ]:
Main_df.head()

### Extract the seasons from started_at

In [ ]:
Main_df['season'] = (Main_df['started_at'].dt.month%12 + 3)//3

seasons = {
             1: 'Winter',
             2: 'Spring',
             3: 'Summer',
             4: 'Autumn'
}

Main_df['season'] = Main_df['season'].map(seasons)

In [ ]:
Main_df.head()

## #Now the data is clean, we have all the columns we need, and it is ready for analysis.

### Note: At this stage, you can visualize the data using Python, or you can export your data as csv and visualize using Tableau.

In [ ]:
Viz_1 = Main_df.groupby(['member_casual']).count()['ride_id']

In [ ]:
# Plot a pie chart that shows the percentage of total rides between casual riders and members.
print("Members: 1,857,313",
     "\nCasuals: 1,150,000")
plot = Viz_1.plot.pie(autopct='%1.0f%%', figsize=(5, 5))

In [ ]:
Viz_2 = Main_df.groupby(['day', 'member_casual']).count()['ride_id']

In [ ]:
Viz_2

In [ ]:
#Viz_3 = Main_df.groupby(['start_hour']).count()['ride_id']
#Viz_3

In [ ]:
#plt.plot(Viz_3['ride_id'].count(), Viz_3['start_hour'])

In [ ]:
#Viz_4 = Main_df.groupby(['day']).count()['ride_id']

In [ ]:
#x = Viz_4['day']
#y = Viz_4['day'])
#plt.bar(Viz_4['day']), )
#plt.show()

In [ ]:
#Main_df.head()